In [16]:
pip install -r requirements.txt

  Using cached tensorflow-2.18.0-cp312-cp312-win_amd64.whl.metadata (3.3 kB)
Using cached tensorflow-2.18.0-cp312-cp312-win_amd64.whl (7.5 kB)
Note: you may need to restart the kernel to use updated packages.


In [18]:
pip install tf-keras


  Using cached tf_keras-2.18.0-py3-none-any.whl.metadata (1.6 kB)
Using cached tf_keras-2.18.0-py3-none-any.whl (1.7 MB)


In [19]:
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from transformers import DistilBertTokenizerFast, TFDistilBertForSequenceClassification


In [24]:
email_ds = pd.read_csv('./model/datasets/emails_dataset.csv')
email_ds.head()

,text,spam
0,Subject: enron methanol ; meter # : 988291\r\n...,0
1,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,"Subject: photoshop , windows , office . cheap ...",1
4,Subject: re : indian springs\r\nthis deal is t...,0


In [25]:
X_train, X_test, y_train, y_test = train_test_split(email_ds['text'], email_ds['spam'], test_size=0.2, random_state=42)

In [22]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
tokenizer.save_pretrained("./model/email_spam_model")

('email_spam_model\\tokenizer_config.json',
 'email_spam_model\\special_tokens_map.json',
 'email_spam_model\\vocab.txt',
 'email_spam_model\\added_tokens.json',
 'email_spam_model\\tokenizer.json')

In [27]:
train_encodings = tokenizer(X_train.tolist(), truncation=True, padding=True)
test_encodings = tokenizer(X_test.tolist(), truncation=True, padding=True)

In [29]:
# Create PyTorch Dataset
class EmailDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

In [45]:
train_dataset = EmailDataset(train_encodings, y_train.tolist())
test_dataset = EmailDataset(test_encodings, y_test.tolist())
test_dataset[0]

{'input_ids': tensor([  101,  3395,  1024,  2128,  1024,  2034,  6959,  1011, 16060, 22684,
          1041,  1004,  1052,  2047,  3066,  2193,  2003, 22777, 18139,  2487,
          6294,  3044,  1030, 14925,  2102,  6021,  1013,  2676,  1013,  2456,
          6185,  1024,  2484,  7610,  2000,  1024,  3419, 28794,  1013, 13058,
          1013,  4372,  4948,  1030,  4372,  4948,  1010,  7628, 11527,  2015,
          1013,  7570,  2226,  1013, 14925,  2102,  1030, 14925,  2102, 10507,
          1024,  3395,  1024,  2128,  1024,  2034,  6959,  1011, 16060, 22684,
          1041,  1004,  1052,  3419,  1024,  2052,  2017,  3531,  2275,  2039,
          2178,  2028,  3204,  7281,  2005,  1996,  2917,  1006,  2197,  3204,
          2001, 19235, 28311,  2509,  1007,  1029,  7628,  1024,  2566,  9606,
          1005,  1055,  3602,  3531,  2079,  2025,  4604,  2041,  2151,  3259,
          2006,  2023,  1012,  2057,  5987,  1996,  3259,  2306,  1037,  2154,
          2030,  2061,  1012,  3531,  2

In [32]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./model/results',
    num_train_epochs=2,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

In [33]:
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

trainer.train()

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  1%|          | 10/1090 [01:08<2:05:03,  6.95s/it]

{'loss': 0.7018, 'grad_norm': 2.616570472717285, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.02}


  2%|▏         | 20/1090 [02:23<2:13:03,  7.46s/it]

{'loss': 0.6823, 'grad_norm': 1.4500535726547241, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.04}


  3%|▎         | 30/1090 [03:38<2:11:28,  7.44s/it]

...

100%|██████████| 1090/1090 [2:12:45<00:00,  6.24s/it]

{'loss': 0.0004, 'grad_norm': 0.005734020844101906, 'learning_rate': 0.0, 'epoch': 2.0}


100%|██████████| 1090/1090 [2:12:47<00:00,  7.31s/it]

{'train_runtime': 7967.2003, 'train_samples_per_second': 2.189, 'train_steps_per_second': 0.137, 'train_loss': 0.0989107018135864, 'epoch': 2.0}


TrainOutput(global_step=1090, training_loss=0.0989107018135864, metrics={'train_runtime': 7967.2003, 'train_samples_per_second': 2.189, 'train_steps_per_second': 0.137, 'total_flos': 2309966497763328.0, 'train_loss': 0.0989107018135864, 'epoch': 2.0})

In [34]:
trainer.evaluate()

100%|██████████| 137/137 [03:25<00:00,  1.50s/it]


{'eval_loss': 0.017079537734389305,
 'eval_runtime': 207.1078,
 'eval_samples_per_second': 10.526,
 'eval_steps_per_second': 0.661,
 'epoch': 2.0}

In [35]:
trainer.predict(test_dataset)

100%|██████████| 137/137 [03:32<00:00,  1.55s/it]


PredictionOutput(predictions=array([[ 4.91339  , -4.7248683],
       [-3.5898714,  3.8752549],
       [-3.5908644,  3.9313364],
       ...,
       [-3.383893 ,  3.6332774],
       [ 2.9832778, -2.8182068],
       [ 4.9017305, -4.6804433]], dtype=float32), label_ids=array([0, 1, 1, ..., 1, 0, 0], dtype=int64), metrics={'test_loss': 0.017079537734389305, 'test_runtime': 214.3257, 'test_samples_per_second': 10.171, 'test_steps_per_second': 0.639})

In [36]:
output=trainer.predict(test_dataset)[1]

100%|██████████| 137/137 [03:24<00:00,  1.49s/it]


In [37]:
from sklearn.metrics import confusion_matrix

cm=confusion_matrix(y_test,output)
cm

array([[1596,    0],
       [   0,  584]], dtype=int64)

In [42]:
import numpy as np
prediction_output = trainer.predict(test_dataset)
predicted_labels = np.argmax(prediction_output.predictions, axis=-1)


100%|██████████| 137/137 [03:15<00:00,  1.43s/it]


In [43]:
from sklearn.metrics import accuracy_score, f1_score
acc = accuracy_score(prediction_output.label_ids, predicted_labels)
f1 = f1_score(prediction_output.label_ids, predicted_labels, average='weighted')
print(f"Accuracy: {acc:.2f}, F1: {f1:.2f}")

Accuracy: 0.99, F1: 0.99


In [44]:
# Evaluate on the test dataset (which contains labels)
eval_results = trainer.evaluate(eval_dataset=test_dataset)
print(eval_results)


100%|██████████| 137/137 [03:06<00:00,  1.36s/it]

{'eval_loss': 0.017079537734389305, 'eval_runtime': 187.754, 'eval_samples_per_second': 11.611, 'eval_steps_per_second': 0.73, 'epoch': 2.0}


In [48]:
trainer.save_model('./model/email_spam_model')

In [23]:
pip install huggingface_hub

Note: you may need to restart the kernel to use updated packages.


In [24]:
from huggingface_hub import login

login(token="token")  # Lấy token từ https://huggingface.co/settings/tokens

In [25]:
from huggingface_hub import upload_folder

repo_id = "jamesnq/email_spam_model"  # Tên repo trên Hugging Face
local_folder = "email_spam_model"  # Thư mục chứa model đã cập nhật

upload_folder(folder_path=local_folder, repo_id=repo_id, repo_type="model")


training_args.bin: 100%|██████████| 5.18k/5.18k [00:01<00:00, 4.79kB/s]


CommitInfo(commit_url='https://huggingface.co/jamesnq/email_spam_model/commit/dda7c1ea14d7bbb37cf0d088ad8ea5db63bfafbe', commit_message='Upload folder using huggingface_hub', commit_description='', oid='dda7c1ea14d7bbb37cf0d088ad8ea5db63bfafbe', pr_url=None, repo_url=RepoUrl('https://huggingface.co/jamesnq/email_spam_model', endpoint='https://huggingface.co', repo_type='model', repo_id='jamesnq/email_spam_model'), pr_revision=None, pr_num=None)